In [45]:
import pandas as pd
import numpy as np
# from jupyter_utils import jupyter_setup, load_tracker, Participant, get_linregr_matrices
import jupyter_utils as ju
ju.jupyter_setup()
import os
from evaluation_and_tracking import IDPerformanceTracker
import torch
from sklearn.metrics import roc_curve, auc, confusion_matrix, precision_recall_curve
from sklearn.linear_model import LinearRegression, LogisticRegression
from utils.utils import FocalLoss
pd.options.display.float_format = '{:.3f}'.format
from torch.nn import BCELoss
import matplotlib.pyplot as plt
# loss_func = FocalLoss(gamma=0, exclude_outliers=0)
loss_func = BCELoss()


C:\Michi\acoustic_covid_detection\python


In [138]:
# ID_PERFORMANCE_TRACKING = "test_linearRegression.pickle"
# ID_PERFORMANCE_TRACKING = "MILmetadata.pickle"
# ID_PERFORMANCE_TRACKING = "02_MILDDenseNoMetadata.pickle"
# ID_PERFORMANCE_TRACKING = "MILDDenseNoMetadata_2epochs.pickle"
# ID_PERFORMANCE_TRACKING = "noMetadataMIL_4modalities.pickle"

# ID_PERFORMANCE_TRACKING = "noMetadataMIL_v2.pickle"
# ID_PERFORMANCE_TRACKING = "metadata_mil.pickle"
# ID_PERFORMANCE_TRACKING = "baseline_models.pickle"
ID_PERFORMANCE_TRACKING = "correct_metadata.pickle"

use_modalities = [
    "combined_coughs",
    # "combined_speech",
   # "combined_breaths",
   # "combined_vowels"
]

In [139]:
id_performance = IDPerformanceTracker(ID_PERFORMANCE_TRACKING)
id_performance.df = id_performance.load()


# for modality in use_modalities:
id_performance.df = id_performance.df[id_performance.df.rec_type.isin(use_modalities)]

total_eval_data = id_performance.df[id_performance.df.set_type == "eval"]
total_test_data = id_performance.df[id_performance.df.set_type == "test"]

seeds = list(id_performance.df.seed.unique())
recording_types = list(total_eval_data.rec_type.unique())
print(f"Seeds used: {seeds}")
print(f"Recording Types used: {recording_types}")

# make sure, that there are even numbers for all 5 seeds, otherwise it did not record correctly
total_eval_data.seed.value_counts(), total_test_data.seed.value_counts()

Seeds used: [66445511337, 55555555, 3213213211, 215674, 99468865]
Recording Types used: ['combined_coughs']


(55555555       268
 215674         266
 3213213211     265
 66445511337    261
 99468865       261
 Name: seed, dtype: int64,
 66445511337    466
 55555555       466
 3213213211     466
 215674         466
 99468865       466
 Name: seed, dtype: int64)

In [140]:
id_performance.df 

,ID,label,loss,prediction,rec_type,seed,set_type
7055,01OCEf1yB4czsq8ygRoT51s96Ba2,0.000,0.7030194997787476,0.01964813657104969,combined_coughs,66445511337,eval
7056,0gB18cWHioTobGuU3qVE0QV5mBN2,0.000,0.7083379030227661,0.030154181644320488,combined_coughs,66445511337,eval
7057,0Ha52POVIxTKEPqI1eGpIoMHUd52,0.000,0.716586172580719,0.04634113237261772,combined_coughs,66445511337,eval
7058,0HIgO2EhOOW1msCbEw1kC8Qsx6D3,0.000,0.7074978947639465,0.02849840372800827,combined_coughs,66445511337,eval
7059,0KSi2atlmsXNcGMfpDNzIjJSvC23,1.000,0.663898766040802,0.05937809869647026,combined_coughs,66445511337,eval
...,...,...,...,...,...,...,...
10701,ZNTuppbDaCNybeDYGmyjJeYr93s1,0.000,0.7117339968681335,0.03683434799313545,combined_coughs,99468865,test
10702,zOmz0TaNdmQfF2jUFkqUKuNa80m1,0.000,0.7098226547241211,0.033077336847782135,combined_coughs,99468865,test
10703,Zs7tQ487P7hws3ur1TG53aiF0eC2,0.000,0.7026061415672302,0.0188292209059,combined_coughs,99468865,test
10704,Zs8oTVcUPTc8pk9UES7sFEhRDxX2,0.000,0.7011853456497192,0.016012169420719147,combined_coughs,99468865,test


In [141]:
# temp = pd.DataFrame(id_performance.df.copy())
val_metrics = pd.DataFrame({"seed": seeds})
test_metrics = pd.DataFrame({"seed": seeds})
metric_columns = ["auc_roc", "loss", "accuracy", "f1_score", "auc_prec_recall", "precision", "tpr_at_95","tpr", "tnr"]
for metric in metric_columns:
    val_metrics[metric] = None
    test_metrics[metric] = None
test_metrics

,seed,auc_roc,loss,accuracy,f1_score,auc_prec_recall,precision,tpr_at_95,tpr,tnr
0,66445511337,None,None,None,None,None,None,None,None,None
1,55555555,None,None,None,None,None,None,None,None,None
2,3213213211,None,None,None,None,None,None,None,None,None
3,215674,None,None,None,None,None,None,None,None,None
4,99468865,None,None,None,None,None,None,None,None,None


In [142]:
allow_n_missing_recordings=3


print(f"Number of participants before and after removing examples, where more than {allow_n_missing_recordings} modalities were missing (e.g. due to audio quality)")    
for seed in seeds:
    # print("\n\n\n\n\n")
    print(f"##############################   seed: {seed}    ####################################")
    
    eval_data = total_eval_data[total_eval_data.seed == seed]
    test_data = total_test_data[total_test_data.seed == seed]
    eval_ids = eval_data.ID.unique()
    test_ids = test_data.ID.unique()
    
    A_val, y_val, _ = ju.get_linregr_matrices(eval_ids, eval_data, allow_n_missing_recordings=allow_n_missing_recordings)
    A_test, y_test, _ = ju.get_linregr_matrices(test_ids, test_data, allow_n_missing_recordings=allow_n_missing_recordings)
    print(f"Validation set: #ids total: {len(eval_ids)} - after removing {len(A_val)}")    
    print(f"Test set: #ids total: {len(test_ids)} - after removing {len(A_test)}")
    A_val = ju.extend_linregr_matrx(A_val)
    A_test = ju.extend_linregr_matrx(A_test)
    model = LinearRegression().fit(A_val, y_val)
    
    y_val_pred = model.predict(A_val)
    y_val_pred = np.clip(y_val_pred, 0, 1)   
    val_aucroc, val_acc = ju.get_aucroc(y_val, y_val_pred), ju.get_accuracy(y_val, y_val_pred)
    y_test_pred = model.predict(A_test)
    y_test_pred = np.clip(y_test_pred, 0, 1)   
    test_aucroc, test_acc = ju.get_aucroc(y_test, y_test_pred), ju.get_accuracy(y_test, y_test_pred)
    
    print("Linear Regression Coefficients (cough, speech, breath, vowels), their respective squares and absolute roots")
    print(np.round(model.coef_[:4], 2))    
    print(np.round(model.coef_[4:8], 2))    
    print(np.round(model.coef_[8:], 2))    
    # print(f"##############################   validation set    ####################################")
    
    mat = ju.get_confusion_matrix_parameters(y_val, y_val_pred, verbose=False)
    tpr, _, tnr, _, precision = ju.get_rates_from_confusion_matrix(mat, verbose=False)
    recall = tpr
    f1_score = 2 * (precision * recall) / (precision + recall)
    
    val_metrics.loc[val_metrics.seed == seed, "auc_roc"] = val_aucroc
    val_metrics.loc[val_metrics.seed == seed, "loss"] = loss_func(torch.Tensor([y_val_pred]),torch.Tensor([y_val]))
    val_metrics.loc[val_metrics.seed == seed, "accuracy"] = val_acc
    val_metrics.loc[val_metrics.seed == seed, "precision"] = precision
    val_metrics.loc[val_metrics.seed == seed, "f1_score"] = f1_score
    val_metrics.loc[val_metrics.seed == seed, "auc_prec_recall"] = ju.get_auc_prec_recall(y_val, y_val_pred)
    val_metrics.loc[val_metrics.seed == seed, "tpr_at_95"] = ju.get_tpr_at_sensitivity(y_val, y_val_pred, sensitivity_target=0.95)
    val_metrics.loc[val_metrics.seed == seed, "tpr"] = tpr
    val_metrics.loc[val_metrics.seed == seed, "tnr"] = tnr
    
    # print(f"##############################   test set    ####################################")
    
    mat = ju.get_confusion_matrix_parameters(y_test, y_test_pred, verbose=False)
    tpr, _, tnr, _, precision = ju.get_rates_from_confusion_matrix(mat, verbose=False)
    recall = tpr
    f1_score = 2 * (precision * recall) / (precision + recall)  
    
    test_metrics.loc[test_metrics.seed == seed, "auc_roc"] = test_aucroc
    test_metrics.loc[test_metrics.seed == seed, "loss"] = loss_func(torch.Tensor([y_test_pred]),torch.Tensor([y_test]))
    test_metrics.loc[test_metrics.seed == seed, "accuracy"] = test_acc
    test_metrics.loc[test_metrics.seed == seed, "precision"] = precision
    test_metrics.loc[test_metrics.seed == seed, "f1_score"] = f1_score
    test_metrics.loc[test_metrics.seed == seed, "auc_prec_recall"] = ju.get_auc_prec_recall(y_test, y_test_pred)
    test_metrics.loc[test_metrics.seed == seed, "tpr_at_95"] = ju.get_tpr_at_sensitivity(y_test, y_test_pred, sensitivity_target=0.95)
    test_metrics.loc[test_metrics.seed == seed, "tpr"] = tpr
    test_metrics.loc[test_metrics.seed == seed, "tnr"] = tnr

Number of participants before and after removing examples, where more than 3 modalities were missing (e.g. due to audio quality)
##############################   seed: 66445511337    ####################################
Validation set: #ids total: 261 - after removing 261
Test set: #ids total: 466 - after removing 466
Linear Regression Coefficients (cough, speech, breath, vowels), their respective squares and absolute roots
[148.62   0.     0.     0.  ]
[-498.75    0.      0.      0.  ]
[-34.9   0.    0.    0. ]
##############################   seed: 55555555    ####################################
Validation set: #ids total: 268 - after removing 268
Test set: #ids total: 466 - after removing 466
Linear Regression Coefficients (cough, speech, breath, vowels), their respective squares and absolute roots
[121.43  -0.     0.     0.  ]
[-469.93    0.      0.      0.  ]
[-25.17   0.     0.     0.  ]
##############################   seed: 3213213211    ####################################
Va

In [143]:
cols = ["auc_roc", "accuracy", "f1_score" , "auc_prec_recall", "precision" , "tpr_at_95" , "tpr" , "tnr"]
multiplier = 1


print(f"##############################   validation set    ####################################")
val_stats = val_metrics.apply(pd.to_numeric, errors='coerce')
val_stats[cols] = val_stats[cols] * multiplier
val_stats = val_stats.drop(columns=["seed"])
val_stats = val_stats.describe().transpose()
val_metrics[cols] = val_metrics[cols] * multiplier
display(val_metrics)
display(val_stats[["mean", "std"]].transpose())

print(f"#################################   test set    #######################################")
test_stats = test_metrics.apply(pd.to_numeric, errors='coerce')

test_stats = test_stats.drop(columns=["seed"])
test_stats[cols] = test_stats[cols] * multiplier
test_stats = test_stats.describe().transpose()
test_metrics[cols] = test_metrics[cols] * multiplier
display(test_metrics)
display(test_stats[["mean", "std"]].transpose())

##############################   validation set    ####################################


,seed,auc_roc,loss,accuracy,f1_score,auc_prec_recall,precision,tpr_at_95,tpr,tnr
0,66445511337,0.921,0.270,0.885,0.706,0.828,0.800,0.614,0.632,0.956
1,55555555,0.897,0.345,0.854,0.715,0.787,0.790,0.640,0.653,0.933
2,3213213211,0.910,0.296,0.883,0.730,0.830,0.840,0.631,0.646,0.960
3,215674,0.906,0.667,0.880,0.709,0.773,0.780,0.517,0.650,0.947
4,99468865,0.917,0.326,0.843,0.701,0.806,0.750,0.521,0.657,0.915


,auc_roc,loss,accuracy,f1_score,auc_prec_recall,precision,tpr_at_95,tpr,tnr
mean,0.910,0.381,0.869,0.712,0.805,0.792,0.584,0.648,0.942
std,0.009,0.163,0.019,0.011,0.025,0.033,0.061,0.010,0.018


#################################   test set    #######################################


,seed,auc_roc,loss,accuracy,f1_score,auc_prec_recall,precision,tpr_at_95,tpr,tnr
0,66445511337,0.898,0.229,0.912,0.549,0.711,0.806,0.683,0.417,0.985
1,55555555,0.895,0.241,0.914,0.565,0.678,0.812,0.600,0.433,0.985
2,3213213211,0.903,0.232,0.918,0.596,0.680,0.824,0.583,0.467,0.985
3,215674,0.844,1.298,0.914,0.556,0.668,0.833,0.567,0.417,0.988
4,99468865,0.880,0.659,0.918,0.620,0.680,0.775,0.600,0.517,0.978


,auc_roc,loss,accuracy,f1_score,auc_prec_recall,precision,tpr_at_95,tpr,tnr
mean,0.884,0.532,0.915,0.577,0.683,0.810,0.607,0.450,0.984
std,0.024,0.466,0.003,0.030,0.016,0.022,0.045,0.042,0.004


# loss
distribution of the loss by participant (1 specific seed) <br>
note that there are some extreme outliers that are 100% certain at predicting the wrong class <br>
due to the nature of linear regression, the values are not limited between 0 and 1, so I clipped them. but still predictions are never 0 or 1 from the network outputs (sigmoid activation). If this is the case, the loss is 100.

In [ ]:
loss_func = BCELoss(reduction="none")
val_loss = loss_func(torch.Tensor([y_val_pred]),torch.Tensor([y_val]))
print("loss with and without extreme outliers (loss > 90 meaning e.g. label=0, target=1)")
val_loss.mean(), val_loss[val_loss < 90].mean()

In [ ]:
val_loss2 = np.array(val_loss[val_loss < 10])
len(val_loss[val_loss > 10])

In [ ]:
_ = plt.hist(val_loss, 40)
plt.ylim([0,100])
plt.figure()
_ = plt.hist(val_loss2, 40)

In [ ]:
_ = plt.hist(y_val_pred, 40), plt.ylim([0, 80])

# Get perofrmance for a single rec type
probably outdated, needs to be updated

In [ ]:
# AUCROC and accuracy for validation set and test set before linear regression
# keep in mind that IDs that were excluded for various reasons (e.g. audio quality) were set to have a prediction right in the middle (0.5) which decreases the performance, especially accuracy
for rec_type_idx in range(4): 
    pred = torch.sigmoid(torch.Tensor(A_val))[:, rec_type_idx].numpy()
    print("eval set:   ", ju.get_aucroc(y_val, pred), ju.get_accuracy(y_val, pred))
    pred = torch.sigmoid(torch.Tensor(A_test))[:, rec_type_idx].numpy()
    print("test set:   ", ju.get_aucroc(y_test, pred), ju.get_accuracy(y_test, pred))
    print("#################################################")

In [ ]:
# rec_type = "combined_coughs"
# rec_type = "combined_speech"
rec_type = "combined_breaths"
# rec_type = "combined_vowels"


In [ ]:
id_performance = IDPerformanceTracker(ID_PERFORMANCE_TRACKING)
id_performance.df = id_performance.load()

In [ ]:
id_performance.df = id_performance.df[id_performance.df.rec_type == rec_type]
recording_types = id_performance.df.rec_type.unique()
recording_types, len(id_performance.df)

In [ ]:
eval_data = id_performance.df[id_performance.df.set_type == "eval"]
test_data = id_performance.df[id_performance.df.set_type == "test"]
eval_ids = eval_data.ID.unique()
test_ids = test_data.ID.unique()
len(eval_ids), len(test_ids)

In [ ]:
recording_types = test_data.rec_type.unique()
recording_types

In [ ]:
A_val, y_val, val_ids_filtered = ju.get_linregr_matrices(eval_ids, eval_data, allow_n_missing_recordings=3)
A_test, y_test, test_ids_filtered = ju.get_linregr_matrices(test_ids, test_data, allow_n_missing_recordings=3)
len(val_ids_filtered), len(test_ids_filtered)

In [ ]:
A_val = extend_linregr_matrx(A_val)
A_test = extend_linregr_matrx(A_test)
model = LinearRegression().fit(A_val, y_val)

In [ ]:
y_val_pred = model.predict(A_val)
get_aucroc(y_val, y_val_pred), get_accuracy(y_val, y_val_pred)

In [ ]:
y_test_pred = model.predict(A_test)
get_aucroc(y_test, y_test_pred), get_accuracy(y_test, y_test_pred)

In [ ]:
len(y_test_pred), len(test_ids_filtered)

In [ ]:
parts_test = [Participant(participant_id, test_data, allow_n_missing_recordings=3) for participant_id in test_ids_filtered]
parts_eval = [Participant(participant_id, eval_data, allow_n_missing_recordings=3) for participant_id in val_ids_filtered]

In [ ]:
new_preds_test = np.array(torch.sigmoid(torch.Tensor(np.array([part.get_single_prediction(rec_type, test_data) for part in parts_test]))))
new_preds_val = np.array(torch.sigmoid(torch.Tensor(np.array([part.get_single_prediction(rec_type, eval_data) for part in parts_eval]))))

# preds = [part.get_single_prediction(rec_type, test_data) for part in parts]
labels_test = np.array([part.label for part in parts_test])
labels_eval = np.array([part.label for part in parts_eval])

In [ ]:
for i in np.linspace(-0.3, 0.0, 40):
    print(i)
    print(get_aucroc(labels_eval, new_preds_val+i), " | ", get_accuracy(labels_eval, new_preds_val+i))

In [ ]:
x = -0.046153846153846156
print(get_aucroc(labels_eval, new_preds_val+x), " | ", get_accuracy(labels_eval, new_preds_val+x))
print(get_aucroc(labels_test, new_preds_test+x), " | ", get_accuracy(labels_test, new_preds_test+x))

In [ ]:
for i in np.linspace(-0.3, 0.0, 20):
    # print(x)
    print(get_aucroc(labels_test, new_preds_test+i), " | ", get_accuracy(labels_test, new_preds_test+i))